In [ ]:
import csv
import pandas as pd
from datetime import datetime, timedelta
import os
from time import time

In [ ]:
file_mapped_by_hours = "./data/processing/hours_{0}.csv"
file_mapped_by_days = "./data/processing/days_{0}.csv"